<a href="https://colab.research.google.com/github/Crash-Analysis-Chicago/Chicago/blob/main/Venkatesh/JDS_Venkatesh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [341]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup, SoupStrainer
import requests

In [342]:
from google.colab import drive
drive.mount('../content/drive/')
 

# colab demande de 'uploader' le dataset pour chaque démarrage. Pour utiliser donc facilement le dataset, uploadez le dataset sur Google Drive.
# pour bien réussir cela, suivez les étape ci-dessous
# étape 1: créez un dossier appelé 'Projet-JDS' ou comme vous voulez sur Google Drive et deposez le dataset dans ce dossier 
# 2: executez cette cellule maintenant, si tout va bien, il vous demande de mot de passe
# 3: Choissez un compte google pour générer le mot de passe
# 4: Collez le mot de passe. C'est fait.


#Normalement ce serait bon, sinon, n'hesitez pas à m'écrire

# Commande Bash pour copier le contenu du dossier Projet-JDS dans le même répertoire que ce notebook
!cp -r /content/drive/My\ Drive/Projet-JDS/* /content

Drive already mounted at ../content/drive/; to attempt to forcibly remount, call drive.mount("../content/drive/", force_remount=True).


In [343]:
csv = pd.read_csv('latest_cleaned.csv')
csv.columns

Index(['CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE_EST_I', 'CRASH_DATE',
       'POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
       'TRAFFICWAY_TYPE', 'LANE_CNT', 'ALIGNMENT', 'ROADWAY_SURFACE_COND',
       'ROAD_DEFECT', 'REPORT_TYPE', 'CRASH_TYPE', 'INTERSECTION_RELATED_I',
       'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE', 'DATE_POLICE_NOTIFIED',
       'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO',
       'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE',
       'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I',
       'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'NUM_UNITS',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
       'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',
       'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH',
       'LA

In [344]:
csv.tail()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,DAMAGE,DATE_POLICE_NOTIFIED,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_NO,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,PHOTOS_TAKEN_I,STATEMENTS_TAKEN_I,DOORING_I,WORK_ZONE_I,WORK_ZONE_TYPE,WORKERS_PRESENT_I,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
448624,e481dd27bc76e18b9a627f605d6d62c5750bca3311222a...,JD313387,NaN,07/28/2020 04:00:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,ONE-WAY,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,N,NaN,NaN,"OVER $1,500",07/28/2020 07:19:00 PM,FAILING TO REDUCE SPEED TO AVOID CRASH,FAILING TO REDUCE SPEED TO AVOID CRASH,1553,S,AVERS AVE,1014.0,NaN,NaN,NaN,NaN,NaN,NaN,2,"REPORTED, NOT EVIDENT",1.0,0.0,0.0,0.0,1.0,0.0,0.0,16,3,7,41.859162,-87.721164,POINT (-87.721164123424 41.859161813268)
448625,df74080be8ceeb327edd744a7f34507bc32c3f76a3f00c...,JD314102,NaN,07/29/2020 02:00:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,ONE-WAY,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,N,NaN,Y,"$501 - $1,500",07/29/2020 02:16:00 PM,UNABLE TO DETERMINE,NOT APPLICABLE,641,E,49TH ST,223.0,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0,14,4,7,41.805737,-87.609784,POINT (-87.609784032292 41.805736682295)
448626,f2ad41aeabd38b5c073f887c66ba64118d1c8843395d5c...,JD313908,NaN,07/29/2020 10:50:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),INJURY AND / OR TOW DUE TO CRASH,Y,NaN,NaN,"OVER $1,500",07/29/2020 11:51:00 AM,DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGAT...,NOT APPLICABLE,3198,W,FULLERTON AVE,1413.0,NaN,NaN,NaN,NaN,NaN,NaN,2,"REPORTED, NOT EVIDENT",1.0,0.0,0.0,0.0,1.0,2.0,0.0,10,4,7,41.924824,-87.707553,POINT (-87.707552955117 41.924824383756)
448627,fcc0426860ee85a4e6db60be5572d317452eb63506d0df...,JD314407,NaN,07/29/2020 06:30:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,N,NaN,NaN,"$501 - $1,500",07/29/2020 07:32:00 PM,UNABLE TO DETERMINE,UNABLE TO DETERMINE,940,W,WEED ST,1822.0,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0,18,4,7,41.909896,-87.652772,POINT (-87.65277243899 41.909895530398)
448628,f829a8bd9a1afc120ed6720cdb20a22b1a90b81278002b...,JD105771,NaN,01/06/2020 08:30:00 AM,15,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,ALLEY,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,N,NaN,NaN,"$501 - $1,500",01/06/2020 09:30:00 AM,UNABLE TO DETERMINE,UNABLE TO DETERMINE,1305,S,MICHIGAN AVE,132.0,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,8,2,1,41.865595,-87.623964,POINT (-87.62396444716 41.865594951208)


#Removal of the columns which are not useful
Crash_hour, Crash_day_of_week, Crash_month existent déjà dans la colonne: Crash_date.

---

Location est une melange de latitude et longitude, qui existent déjà dans la colonne: Latitude et longtitude
 

In [345]:
csv = csv.drop(['CRASH_RECORD_ID','RD_NO','CRASH_DATE_EST_I','BEAT_OF_OCCURRENCE','NOT_RIGHT_OF_WAY_I',
              'PHOTOS_TAKEN_I','STATEMENTS_TAKEN_I','REPORT_TYPE','HIT_AND_RUN_I','DATE_POLICE_NOTIFIED','STREET_NO','STREET_DIRECTION','STREET_NAME'],axis=1)

# Removal of the columns that are redundant
csv = csv.drop(['CRASH_HOUR', 'CRASH_DAY_OF_WEEK','CRASH_MONTH'],axis=1)
csv.columns


Index(['CRASH_DATE', 'POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE',
       'DEVICE_CONDITION', 'WEATHER_CONDITION', 'LIGHTING_CONDITION',
       'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 'LANE_CNT', 'ALIGNMENT',
       'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'CRASH_TYPE',
       'INTERSECTION_RELATED_I', 'DAMAGE', 'PRIM_CONTRIBUTORY_CAUSE',
       'SEC_CONTRIBUTORY_CAUSE', 'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE',
       'WORKERS_PRESENT_I', 'NUM_UNITS', 'MOST_SEVERE_INJURY',
       'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING',
       'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
       'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'LATITUDE', 'LONGITUDE',
       'LOCATION'],
      dtype='object')

In [346]:
csv.head()

,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,INTERSECTION_RELATED_I,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,DOORING_I,WORK_ZONE_I,WORK_ZONE_TYPE,WORKERS_PRESENT_I,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,LATITUDE,LONGITUDE,LOCATION
0,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,4.0,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,Y,"OVER $1,500",FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,0.0,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)
1,03/21/2019 10:50:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,4.0,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,Y,"OVER $1,500",UNABLE TO DETERMINE,UNABLE TO DETERMINE,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)
2,03/26/2018 02:23:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,Y,"$501 - $1,500",UNABLE TO DETERMINE,UNABLE TO DETERMINE,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,41.953647,-87.732082,POINT (-87.732081736006 41.953646899951)
3,02/20/2020 04:45:00 PM,35,TRAFFIC SIGNAL,OTHER,CLEAR,DAWN,REAR END,T-INTERSECTION,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,Y,"$501 - $1,500",NOT APPLICABLE,NOT APPLICABLE,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,41.958987,-87.933994,POINT (-87.933993928974 41.958986950953)
4,08/30/2018 05:45:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,UNKNOWN,NO INJURY / DRIVE AWAY,Y,"OVER $1,500",IMPROPER OVERTAKING/PASSING,IMPROPER LANE USAGE,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,41.903825,-87.643286,POINT (-87.643286359995 41.903825233976)


#Est-ce que les colonnes possedent des valeurs nan? Does any of the columns hold any nan values?

In [347]:
print('{}% ont des colonnes ont NaN'.format(round(sum(csv.isna().any())/len(csv.columns)*100,2)))
# comme vous le voyez ci-dessous, après l'enlèvement, il y a vers 40% de colonnes qui possedent des valeurs nan (la vie serait fou donc je suppose!)

39.39% ont des colonnes ont NaN


In [348]:
#sanity check
csv.columns

Index(['CRASH_DATE', 'POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE',
       'DEVICE_CONDITION', 'WEATHER_CONDITION', 'LIGHTING_CONDITION',
       'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 'LANE_CNT', 'ALIGNMENT',
       'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'CRASH_TYPE',
       'INTERSECTION_RELATED_I', 'DAMAGE', 'PRIM_CONTRIBUTORY_CAUSE',
       'SEC_CONTRIBUTORY_CAUSE', 'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE',
       'WORKERS_PRESENT_I', 'NUM_UNITS', 'MOST_SEVERE_INJURY',
       'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING',
       'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
       'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'LATITUDE', 'LONGITUDE',
       'LOCATION'],
      dtype='object')

#De meme, quel est le ***ratio*** de NaN et les valeurs ?

In [349]:
csv_raw = pd.read_csv('/content/Traffic_Crashes_-_Chicago.csv')
# IMPORTANT: to your understanding, I have used the raw/uncleaned csv just to explain you how many in the LATITUDE are null

**Important**: The csv I mount from Google Drive is after executing the code below, why ? I would have explained in the upcoming cells. Check it out. Key Takeaway: the 2441 cells dont have a value in the latitude column.

In [350]:
print(sum(csv_raw['LATITUDE'].isna()))
print('The {}% of cells do not have values'.format(sum(csv_raw['LATITUDE'].isna())/len(csv)*100))
# 2441 cells dont have a value, that approx. less than 1 percent of total dataset size, removing is an option, why am I removing? 2 obvious reasons:
# 1 percent (0.5% to be precise) is too small and the location is the most important piece of data!
 

# the code below is a bit hard to interpret? hear me out: firstly we find where are the null values are being spotted, locate on the dataframe with iloc, get its index then drop it finally.
csv = csv.drop(csv.iloc[((np.where(csv['LATITUDE'].isna().values==True)))].index)
# as the rows are dropped the index would indeed be disorganised, fixing that with reset_index
#csv = csv.reset_index()

2441
The 0.544102142304666% of cells do not have values


In [351]:
# sanity check
sum(csv['LATITUDE'].isna().values)

0

# Combien de colonnes possedent des valeurs NaN?

In [352]:
csv.isna().any()
# la colonne INTERSECTION_RELATED_I est effectivement plein de valeurs nan, pour remplir ces valeurs là, j'avais trouvé un API(application program interface).
# okay, tout simplement, je vais expliquer prochainement dans les cellules suivantes (ce serait plus approprié).
# j'avais rempli "yet_to_clean" to replace nan for our understanding.

CRASH_DATE                       False
POSTED_SPEED_LIMIT               False
TRAFFIC_CONTROL_DEVICE           False
DEVICE_CONDITION                 False
WEATHER_CONDITION                False
LIGHTING_CONDITION               False
FIRST_CRASH_TYPE                 False
TRAFFICWAY_TYPE                  False
LANE_CNT                          True
ALIGNMENT                        False
ROADWAY_SURFACE_COND             False
ROAD_DEFECT                      False
CRASH_TYPE                       False
INTERSECTION_RELATED_I           False
DAMAGE                           False
PRIM_CONTRIBUTORY_CAUSE          False
SEC_CONTRIBUTORY_CAUSE           False
DOORING_I                         True
WORK_ZONE_I                       True
WORK_ZONE_TYPE                    True
WORKERS_PRESENT_I                 True
NUM_UNITS                        False
MOST_SEVERE_INJURY                True
INJURIES_TOTAL                    True
INJURIES_FATAL                    True
INJURIES_INCAPACITATING  

In [353]:
#combien de colonnes portent des valeurs nan
csv.isna().any().values.sum() # 13 en total

13

# On avance à gérer une colonne après autres pour que vous comprenez bien

1.LANE_CNT



 Quel pourcentage des valeurs sont ***nulles*** dans la colonne LANE_CNT ?

In [354]:
print("{}%".format(sum(csv['LANE_CNT'].isna())/len(csv) *100))
#la colonne Lane_CNT a 56% des valeurs nan, on peut remplir avec moyen valeurs, mais ce n'est pas utile. 
#en considerant de l'importance de la description de cette colonne et les nombreux des nan valeurs, je vais l'enlever

#this feature has around 56% nan values, though we can fill up with values such as mean, this would be futile.

#Besides, considering the description of this feature, I feel removing this whole column is the best way possible.
csv = csv.drop(['LANE_CNT'],axis=1)

55.85171712038232%


2.DOORING_I





In [355]:
#DOORING_I : Whether crash involved a motor vehicle occupant opening a door into the travel path of a bicyclist, causing a crash
# A mon avis cette colonne est au milieu de deux: utile vs inutile, donc je ne l'avais pas enlevée.

# c'est de trouver qu'est-ce qu'elle contient?
print('{}% des valeurs sont nulles'.format(csv['DOORING_I'].isna().sum()/len(csv)*100))

#enlèvement
csv = csv.drop(['DOORING_I'],axis=1)

99.66609381025302% des valeurs sont nulles


3.WORK_ZONE_I: "Whether the crash occurred in an active work zone".
A mon avis, cette colonne est importante, mais, voyons voir combien sont NaN.

In [356]:
# Il semble que WORK_ZONE_I ait des valeurs manquant, mais combien ?
print('{}%'.format(csv['WORK_ZONE_I'].isna().sum()/len(csv)*100)) #99.3%
# enlèvement
csv = csv.drop('WORK_ZONE_I',axis=1)
#sanity check
print(np.unique(csv.columns == 'WORK_ZONE_I'))

99.33441663378872%
[False]


Les suivantes 2 ( **4 & 5**)
WORKERS_ZONE_TYPE, WORKERS_PRESENT_I sont pertinent lorsque WORK_ZONE_I existe. En tant qu'on vient de l'enlever, ces deux sont intuile




In [357]:
#enlèvement
csv = csv.drop(['WORK_ZONE_TYPE','WORKERS_PRESENT_I'],axis=1)

In [358]:
#on a descendu trop, donc il vaut mieux si on voit quelles sont les colonnes existent toujours a gérer pour qu'on n'a pas besoin de monter pas trop. 
csv.isna().any()

CRASH_DATE                       False
POSTED_SPEED_LIMIT               False
TRAFFIC_CONTROL_DEVICE           False
DEVICE_CONDITION                 False
WEATHER_CONDITION                False
LIGHTING_CONDITION               False
FIRST_CRASH_TYPE                 False
TRAFFICWAY_TYPE                  False
ALIGNMENT                        False
ROADWAY_SURFACE_COND             False
ROAD_DEFECT                      False
CRASH_TYPE                       False
INTERSECTION_RELATED_I           False
DAMAGE                           False
PRIM_CONTRIBUTORY_CAUSE          False
SEC_CONTRIBUTORY_CAUSE           False
NUM_UNITS                        False
MOST_SEVERE_INJURY                True
INJURIES_TOTAL                    True
INJURIES_FATAL                    True
INJURIES_INCAPACITATING           True
INJURIES_NON_INCAPACITATING       True
INJURIES_REPORTED_NOT_EVIDENT     True
INJURIES_NO_INDICATION            True
INJURIES_UNKNOWN                  True
LATITUDE                 

 6.MOST_SEVERE_INJURY:

In [359]:
print('{} cells ont des valeurs NaN'.format(csv['MOST_SEVERE_INJURY'].isna().sum()))

#enlèvement
csv = csv.drop(csv.iloc[((np.where(csv['MOST_SEVERE_INJURY'].isna().values==True)))].index)

892 cells ont des valeurs NaN


UPDATE(noticed minutes before commiting to github): En fait, enlevant les valeurs nulls dans la colonne MOST_SEVERE_INJURY a un effet sur les suivantes: INJURIES_TOTAL, etc. I keep the following cells for you to understand how I handle the missing values in categorical type of data.

7.INJURIES_TOTAL

In [360]:
# dans cette cellule, je vais gérer des valeurs manquant dans la colonne INJURIES_TOTAL.

#combien sont nulles?
print('{}%\n'.format(csv['INJURIES_TOTAL'].isna().sum()/len(csv)*100)) # 0.2% pourcent sont nulles, c'est bon, mais les valeurs sont facilement remplis, comment?

#INJURIES_TOTAL- implique combien de personnes sont touché
# Je constate que, INJURIES_TOTAL est relié a la colonne 'damage'.

#combien de types de damage?
print(csv['DAMAGE'].value_counts()) # 3types, a priori, il semble que le plus cher le dommage, les nombreux de mondes sont touché. Mais il faut confirmer.


0.0%

OVER $1,500      256567
$501 - $1,500    132560
$500 OR LESS      58610
Name: DAMAGE, dtype: int64


In [361]:
# verifier combien de mondes dont 'damage' est au-dela $1500 sont touché
print('Over $1500:\n',(csv['INJURIES_TOTAL'][csv['DAMAGE']=='OVER $1,500']).value_counts()) # beaucoup de 0s !

Over $1500:
 0.0     217097
1.0      27696
2.0       7718
3.0       2541
4.0        926
5.0        341
6.0        138
7.0         55
8.0         19
9.0         14
10.0         8
11.0         3
15.0         3
13.0         2
21.0         2
12.0         2
16.0         1
19.0         1
Name: INJURIES_TOTAL, dtype: int64


In [362]:
# vérifier combien de mondes dont 'damage' est entre $501 - $1500 sont touché
print('entre $501 - $1500:\n',(csv['INJURIES_TOTAL'][csv['DAMAGE']=='$501 - $1,500']).value_counts())

entre $501 - $1500:
 0.0     124571
1.0       6592
2.0       1019
3.0        254
4.0         74
5.0         33
6.0         11
7.0          2
11.0         2
15.0         1
10.0         1
Name: INJURIES_TOTAL, dtype: int64


In [363]:
# vérifier combien de mondes, dont 'damage' est entre '$500 OR LESS', sont touché
print('$500 OR LESS:\n',(csv['INJURIES_TOTAL'][csv['DAMAGE']=='$501 - $1,500']).value_counts())


# en considerant les 3 dernieres cellules, cela implique clairement que les valeurs ont beaucoup de variance, donc
# pour conclure: 'damage' et 'injuries_total' ne sont pas correlés. Et considerant que les valeurs NaN ne constituent que 0.2%, je vais l'enlever les 'rows'.

csv = csv.drop(csv.iloc[((np.where(csv['INJURIES_TOTAL'].isna().values==True)))].index)
#sanity check
len(csv) #parfait

$500 OR LESS:
 0.0     124571
1.0       6592
2.0       1019
3.0        254
4.0         74
5.0         33
6.0         11
7.0          2
11.0         2
15.0         1
10.0         1
Name: INJURIES_TOTAL, dtype: int64


447737

Status Check

In [364]:
csv.isna().any()
# vous vous demandez qu'on n'avait fini que gérer 7 colonnes mais toutes les colonnes sont propres, comment cela possible? rep: 'INJURIES_TOTAL', 
# the other columns which has had null values in the columns are as same as the INJURIES_TOTAL. 
# Simply put, removing the null values in INJURIES_TOTAL removes the nulls in all other related columns too. Because it is 'TOTAL'.

CRASH_DATE                       False
POSTED_SPEED_LIMIT               False
TRAFFIC_CONTROL_DEVICE           False
DEVICE_CONDITION                 False
WEATHER_CONDITION                False
LIGHTING_CONDITION               False
FIRST_CRASH_TYPE                 False
TRAFFICWAY_TYPE                  False
ALIGNMENT                        False
ROADWAY_SURFACE_COND             False
ROAD_DEFECT                      False
CRASH_TYPE                       False
INTERSECTION_RELATED_I           False
DAMAGE                           False
PRIM_CONTRIBUTORY_CAUSE          False
SEC_CONTRIBUTORY_CAUSE           False
NUM_UNITS                        False
MOST_SEVERE_INJURY               False
INJURIES_TOTAL                   False
INJURIES_FATAL                   False
INJURIES_INCAPACITATING          False
INJURIES_NON_INCAPACITATING      False
INJURIES_REPORTED_NOT_EVIDENT    False
INJURIES_NO_INDICATION           False
INJURIES_UNKNOWN                 False
LATITUDE                 

In [365]:
# Voyons voir les mots magiques
print('{}% ont des colonnes ont NaN'.format(round(sum(csv.isna().any())/len(csv.columns)*100,2))) #C'EST BON! #ALL CAPS!: DONALD TRUMP'S INSPIRATIONS :D

0.0% ont des colonnes ont NaN


In [366]:
csv.tail()

,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,INTERSECTION_RELATED_I,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,LATITUDE,LONGITUDE,LOCATION
448624,07/28/2020 04:00:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,N,"OVER $1,500",FAILING TO REDUCE SPEED TO AVOID CRASH,FAILING TO REDUCE SPEED TO AVOID CRASH,2,"REPORTED, NOT EVIDENT",1.0,0.0,0.0,0.0,1.0,0.0,0.0,41.859162,-87.721164,POINT (-87.721164123424 41.859161813268)
448625,07/29/2020 02:00:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,N,"$501 - $1,500",UNABLE TO DETERMINE,NOT APPLICABLE,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0,41.805737,-87.609784,POINT (-87.609784032292 41.805736682295)
448626,07/29/2020 10:50:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,Y,"OVER $1,500",DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGAT...,NOT APPLICABLE,2,"REPORTED, NOT EVIDENT",1.0,0.0,0.0,0.0,1.0,2.0,0.0,41.924824,-87.707553,POINT (-87.707552955117 41.924824383756)
448627,07/29/2020 06:30:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,N,"$501 - $1,500",UNABLE TO DETERMINE,UNABLE TO DETERMINE,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0,41.909896,-87.652772,POINT (-87.65277243899 41.909895530398)
448628,01/06/2020 08:30:00 AM,15,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,ALLEY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,N,"$501 - $1,500",UNABLE TO DETERMINE,UNABLE TO DETERMINE,2,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,41.865595,-87.623964,POINT (-87.62396444716 41.865594951208)


#Code pour exporter le dataset nettoyé (sans avoir nettoyé complètement la colonne INTERSECTION_RELATED_I)


In [372]:
#Merci d'executer cette cellule et regarder les paramètres a gauche extrème. Le dataset y serait.
csv_export = csv.to_csv('properly_cleaned_dataset_chicago.csv',index=False)

# IMPORTANT: Je n'ai pas avancé sur INTERSECTION_RELATED_I en tant que je n'avais pas trouvé une bonne moyenne (efficace) de les remplir. Au pire, on évite de le parler dans cette présentation et la remplit après. 

#A propos de ***gestion*** des valeurs manquant dans la colonne 'INTERSECTION_RELATED_I'.

In [ ]:
print('{}% of values are null'.format(round(sum(csv['INTERSECTION_RELATED_I'].isna())/len(csv)*100),2))
# Road intersection is one of the extremely important criteria, but there more than 78% null values!. Crazy indeed. Challenging & Interesting task. 
# Note : The result below would be 0%. That is because of the fact that I filled with "yet_to_fill" for monitoring where I left off.
# Must not be dropped and cannot be filled randomly. So what ?

# Now it is the time to explain how I handle the missing values in INTERSECTION_SECTION_I. 

# After couple of hours of research I found out a nice website geonames.org, which provide the solution to address the problem
# which problem?!: handling the missing data (categorical) related to intersection of the road where the accident took place nearby.
# How does this website help?: this helps in identifying whether there is any intersection close by(100 meters I believe) using the coordinates.
# How to do it? they provide a web-based solution to find that. So that is basically a web scraping to find out the result given the coordinates.
# How to web-scrap it? Beautifulsoup. 

0% of values are null


# Here me out the complex part. Modus-operandi: 


# 1. I repeat. The goal is to find out whether the INTERESECTION_RELATED is true or not, so what do we need? the coordinates. There is no point in finding the result when there is already data. So I take only the coordinates only when the INTERESECTION_RELATED is NaN. Combine the latitude and longitude.
# 2. Make URLs out of the coordinates.
# 3. Pass it as http request the each URLs
# 4. Parse the http response.
# 5. Look for whether there is any data related to intersection, fill 'Y' if yes, fill 'N' if no.



The caveat of this approach:
1. There are limitations such as, restriction on the number of request i.e 1000/hour. Meaning, the api automatically throws out an error message when we exceed 1000 INTERESECTION_RELATED cells. Indeed, it is not feasible if it is the case. I have started finding out work-around. Will update.
2. Problem on execution time: parsing is half a problem but the rest is on Colab. Why though? simply put, the code we run firstly go to Colab server then  as url (request), so it takes a considerable amount of time. Cleaning on a local machine is the best option, so I took this route. To make you aware of what is the code, I have provided the code below. 

In [ ]:
#filling of missing data
csv['INTERSECTION_RELATED_I'] = csv['INTERSECTION_RELATED_I'].fillna(0)

In [ ]:
nan_index = np.where(csv['INTERSECTION_RELATED_I'].values==0)

In [ ]:
def webscraping(latitude,longitude):
    a = "http://api.geonames.org/findNearestIntersection?lat={}&lng={}&username=venk".format(latitude,longitude)

    page = requests.get(a)
    bsp = BeautifulSoup(page.content,'html.parser')
    return bsp.prettify().split()[6]=='<intersection>'


In [368]:
%%time
mm =[]
for i in range(10):
  
  csv['INTERSECTION_RELATED_I'].iloc[nan_index[0][i]]=(webscraping(csv['LATITUDE'][i],csv['LONGITUDE'][i]))